<a href="https://colab.research.google.com/github/Mohammed287-lab/ANN/blob/main/olive_leaf_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#file save

In [1]:
from google.colab import files

uploaded = files.upload()  # ارفع dataset.zip


Saving archive (13).zip to archive (13).zip


In [2]:
import zipfile
import os

zip_path = "archive (13).zip"   # اسم الملف المرفوع
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("تم فك الضغط بنجاح")


تم فك الضغط بنجاح


In [3]:
for root, dirs, files in os.walk("/content/dataset"):
    level = root.replace("/content/dataset", "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")


dataset/
    dataset/
        train/
            Healthy/
            aculus_olearius/
            olive_peacock_spot/
        test/
            Healthy/
            aculus_olearius/
            olive_peacock_spot/


#pre

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    "/content/dataset/dataset/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    "/content/dataset/dataset/test",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)


Found 2720 images belonging to 3 classes.
Found 680 images belonging to 3 classes.


#model


##save_model

In [ ]:
model.save("my_model.h5")
